In [1]:
import os
import pandas as pd
import numpy as np
#import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import datasets
from collections import Counter

from joblib import dump
from sklearn.externals import joblib

#from keras.models import model_from_json

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


C:\Users\a.larav\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Se importa el Dataset con informacion de nubes
Como se ha descrito, las imagenes descargadas de los portales de imagenes satelitales presentan información en multiples bandas. Es con esta información que los algoritmos de clasificación pueden emitir un juicio para determinar a que tipo de información puede corresponder un pixel: bosque, no busque,nubes,agua, agua_bosque,ninguno.
En este escenario se plantea probar el clasificador Naive Bayes con información de imagenes que contengan nubes y agregar una nueva clase: Nubes. 
Este escenario se plantea en respuesta a que la clasificación de imagenes satelitales cuyas nubes fueron removidas resultaban un problema facil de clasificar para este tipo de modelo. 

In [2]:
path = './satellite_dataset-con-nubes.csv'

satellite = pd.read_csv(path)
satellite.head(100)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,cloud,ninguno
0,286.0,197.0,96.0,83.0,116.0,95.0,1,0,0,0,0
1,280.0,217.0,108.0,83.0,118.0,96.0,1,0,0,0,0
2,3011.0,2996.0,2900.0,4186.0,1683.0,1373.0,0,0,0,1,0
3,3576.0,3303.0,3313.0,3369.0,2637.0,2339.0,0,0,0,1,0
4,1398.0,1345.0,1311.0,1389.0,1633.0,1355.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
95,1720.0,1848.0,1722.0,1650.0,1838.0,1476.0,0,0,0,1,0
96,584.0,456.0,402.0,405.0,389.0,356.0,0,0,0,0,1
97,5492.0,5736.0,5891.0,7423.0,6190.0,4758.0,0,0,0,1,0
98,229.0,139.0,43.0,29.0,73.0,59.0,1,0,0,0,0


# Filtro de valores negativos
Se realiza una filtro de las columnas que en alguna banda contengan un valor negativo porque el clasificador multinomial Naive bayes no funciona bien con esta información.

In [3]:
satellite = satellite[satellite["blue"] > 0]
satellite = satellite[satellite["red"] > 0]
satellite = satellite[satellite["green"] > 0]
satellite = satellite[satellite["nir"] > 0]
satellite = satellite[satellite["swir1"] > 0]
satellite = satellite[satellite["swir2"] > 0]

In [4]:
#satellite['wofs'].value_counts()
#satellite['bosque'].value_counts()
#satellite['wofs_bosque'].value_counts()
#satellite['cloud'].value_counts()
#satellite['ninguno'].value_counts()

# Se reemplaza los valores en columnas de clases
Como parte de preparación de los datos para crear el modelo se deben hacer algunos ajustes a la forma en que el dataset muestra la infiromación.Como se puede observar el dataset, las columnas de las clases se encuentran categorizadas por un valor binario. Lo que se realiza a continuación es reemplazar estos valores binarios por NaN en caso de ser "Cero" y el nombre de la clase en caso de ser "Uno".

In [5]:
# Reemplazar el valor de columna Wofs por NaN(0) y Wofs(1)
satellite['wofs'] = satellite['wofs'].replace(0,np.nan)
satellite['wofs'] = satellite['wofs'].replace(1,'wofs')


In [6]:
# Reemplazar el valor de columna bosque por NaN(0) y bosque(1)
satellite['bosque'] = satellite['bosque'].replace(0,np.nan)
satellite['bosque'] = satellite['bosque'].replace(1,'bosque')

In [7]:
# Reemplazar el valor de columna wofs_bosque por NaN(0) y wofs_bosque(1)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(0,np.nan)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(1,'wofs_bosque')

In [8]:
# Reemplazar el valor de columna ninguno por NaN(0) y ninguno(1)
satellite['ninguno'] = satellite['ninguno'].replace(0,np.nan)
satellite['ninguno'] = satellite['ninguno'].replace(1,'ninguno')

In [9]:
# Reemplazar el valor de columna ninguno por NaN(0) y cloud(1)
satellite['cloud'] = satellite['cloud'].replace(0,np.nan)
satellite['cloud'] = satellite['cloud'].replace(1,'cloud')

# Nueva columna de clasificación
Como ultima parte para la preparación de los datos debemos tener la clasificación en una sola columna. Procedemos a crearla con el nombre "label" y alli asignaremos el valor categorizado de cada conjunto de atributos.

In [10]:
#Se crea una nueva columna para almacenar el valor de etiqueta
satellite['label']=np.nan

#Se indica en la columna 'label', la clasificacion de la columna 'wofs'. Se esribe "Wofs" en columna 'label' si en  
#la columna 'wofs' esta este valor
for label in [col for col in satellite.columns if 'wofs' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'bosque'. Se esribe "bosque" en columna 'label' si en  
#la columna 'bosque' esta este valor
for label in [col for col in satellite.columns if 'bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'wofs_bosque'. Se esribe "wofs_bosque" en columna 'label' si en  
#la columna 'wofs_bosque' esta este valor
for label in [col for col in satellite.columns if 'wofs_bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'ninguno'. Se esribe "ninguno" en columna 'label' si en  
#la columna 'ninguno' esta este valor
for label in [col for col in satellite.columns if 'ninguno' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'cloud'. Se esribe "cloud" en columna 'label' si en  
#la columna 'cloud' esta este valor
for label in [col for col in satellite.columns if 'cloud' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
satellite.head(20)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,cloud,ninguno,label
0,286.0,197.0,96.0,83.0,116.0,95.0,wofs,NaN,NaN,NaN,NaN,wofs
1,280.0,217.0,108.0,83.0,118.0,96.0,wofs,NaN,NaN,NaN,NaN,wofs
2,3011.0,2996.0,2900.0,4186.0,1683.0,1373.0,NaN,NaN,NaN,cloud,NaN,cloud
3,3576.0,3303.0,3313.0,3369.0,2637.0,2339.0,NaN,NaN,NaN,cloud,NaN,cloud
4,1398.0,1345.0,1311.0,1389.0,1633.0,1355.0,NaN,NaN,NaN,NaN,ninguno,ninguno
5,328.0,232.0,118.0,96.0,124.0,95.0,wofs,NaN,NaN,NaN,NaN,wofs
6,230.0,203.0,105.0,96.0,129.0,105.0,wofs,NaN,NaN,NaN,NaN,wofs
7,218.0,317.0,142.0,125.0,169.0,150.0,wofs,NaN,NaN,NaN,NaN,wofs
8,6702.0,6782.0,7018.0,7304.0,6323.0,5154.0,NaN,NaN,NaN,cloud,NaN,cloud
9,2866.0,2807.0,2806.0,2901.0,3267.0,2668.0,NaN,NaN,NaN,cloud,NaN,cloud


wofs
cloud
ninguno
boque
wofs_bosque

In [11]:
# Se crea el conjunto de clasificacion y el de entrenamiento
Y = satellite["label"]
X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","cloud","ninguno"], axis=1)
Y.value_counts()

cloud          538313
wofs           288423
bosque          99236
ninguno         61378
wofs_bosque       300
Name: label, dtype: int64

# Se crean los conjuntos de entrenamiento y de prueba
Para hacer esto se deben dividir los datos en 4 conjuntos de datos de la siguiente manera: Conjuntos de entrenamiento(X_train, y_train) y Conjunto de test(X_test, y_test)

In [12]:
# Se dividen los datos en los conjuntos de Train y test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=0
)

In [13]:
X_test

,blue,green,red,nir,swir1,swir2
441018,4415.0,4694.0,4728.0,4888.0,4886.0,4312.0
738664,3464.0,3533.0,3523.0,4645.0,3161.0,2128.0
472244,393.0,334.0,238.0,220.0,319.0,257.0
574853,5000.0,5063.0,5114.0,5233.0,5074.0,4151.0
434050,2950.0,2903.0,2836.0,3711.0,2514.0,1761.0
...,...,...,...,...,...,...
518086,430.0,299.0,187.0,148.0,159.0,130.0
950980,112.0,293.0,162.0,1484.0,748.0,378.0
712658,4317.0,4643.0,4346.0,4060.0,2553.0,1465.0
921580,1513.0,1520.0,1432.0,1400.0,1641.0,1449.0


In [14]:
y_test

441018     cloud
738664     cloud
472244      wofs
574853     cloud
434050     cloud
           ...  
518086      wofs
950980    bosque
712658     cloud
921580     cloud
234064     cloud
Name: label, Length: 296295, dtype: object

# Naive Bayes para multiples capas
La clasificación multicapa del algoritmo Naive Bayes es conocida como "Clasificación Naive Bayes Multinomial". Como se puede apreciar la clasificación de nuestro conjunto de caracteristicas puede tener 4 valores: wofs, bosque, wofs_bosque y ninguno. 
En esta parte de creación del modelo se evaluaran dos variaciones del clasificador Naive Bayes: Gausiano y multinomial. En el analisis de resultados se contrastaran los resultados obtenidos para cada uno de ellos.

### Multinomial

In [15]:
#Create a Gaussian Classifier
gnb = MultinomialNB()

In [16]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [18]:
Counter(y_pred)

Counter({'ninguno': 45250,
         'cloud': 133091,
         'wofs': 83475,
         'bosque': 30659,
         'wofs_bosque': 3820})

In [19]:
#wofs
#cloud
#ninguno
#bosque
#wofs_bosque
confusion_matrix(y_test, y_pred)

array([[ 28721,      1,    320,      2,    551],
       [    43, 120350,  39851,    240,   1295],
       [  1895,   8902,   4897,   1003,   1646],
       [     0,   3826,    175,  82221,    278],
       [     0,     12,      7,      9,     50]], dtype=int64)

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.94      0.97      0.95     29595
       cloud       0.90      0.74      0.82    161779
     ninguno       0.11      0.27      0.15     18343
        wofs       0.98      0.95      0.97     86500
 wofs_bosque       0.01      0.64      0.03        78

    accuracy                           0.80    296295
   macro avg       0.59      0.71      0.58    296295
weighted avg       0.88      0.80      0.83    296295



In [21]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7973101132317454


In [22]:
joblib.dump(gnb, 'model_nb_multinomial.joblib') 

['model_nb_multinomial.joblib']

### Gaussiano

In [23]:
#Create a Gaussian Classifier
gnb = GaussianNB()

In [24]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [26]:
Counter(y_pred)

Counter({'cloud': 135465,
         'wofs': 84790,
         'bosque': 29890,
         'ninguno': 44934,
         'wofs_bosque': 1216})

In [27]:
#wofs
#cloud
#ninguno
#bosque
#wofs_bosque
confusion_matrix(y_test, y_pred)

array([[ 27133,     74,   1321,    349,    718],
       [    68, 134215,  27495,      1,      0],
       [  2570,   1176,  13427,   1065,    105],
       [    95,      0,   2685,  83335,    385],
       [    24,      0,      6,     40,      8]], dtype=int64)

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.91      0.92      0.91     29595
       cloud       0.99      0.83      0.90    161779
     ninguno       0.30      0.73      0.42     18343
        wofs       0.98      0.96      0.97     86500
 wofs_bosque       0.01      0.10      0.01        78

    accuracy                           0.87    296295
   macro avg       0.64      0.71      0.65    296295
weighted avg       0.94      0.87      0.89    296295



In [29]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8711520612902681


In [30]:
joblib.dump(gnb, 'model_nb_gaussiano.joblib') 

['model_nb_gaussiano.joblib']

## Analisis de resultados y conclusiones

El rendmiento obtenido con el modelo Multinomial llega a ser del 79.73% y el rendimiento con el modelo Gausiano alcanza el 87.11%. Por lo que logramos determinar que en este escenario funciona mejor el modelo Gassiano que el Multinomial por aproimadamente 8 puntos de diferencia. Revisando la matriz de confusión logramos evidenciar que el modelo Multinomial confunde mucho de los elementos de la clase 3(Ninguno) siendo esto logico porque los valores de caracteristicas de entrada.

**Multinomial**

**Accuracy:** 79.73%

**Matriz de predicción**

| Clase\Predicción | WOFS  | Cloud  | Ninguno | Bosque | WOFS_Bosque |
|------------------|-------|--------|---------|--------|-------------|
| WOFS             | 28721 | 1      | 320     | 2      | 551         |
| Cloud            | 43    | 120350 | 39851   | 240    | 1295        |
| Ninguno          | 1895  | 8902   | 4897    | 1003   | 1646        |
| Bosque           | 0     | 3826   | 175     | 82221  | 278         |
| WOFS_Bosque      | 0     | 12     | 7       | 9      | 50          |
       
**Gaussiano**

**Accuracy:** 87.11%

**Matriz de predicción**

| Clase\Predicción | WOFS  | Cloud  | Ninguno | Bosque | WOFS_Bosque |
|------------------|-------|--------|---------|--------|-------------|
| WOFS             | 27133 | 74     | 1321    | 349    | 718         |
| Cloud            | 68    | 134215 | 27495   | 1      | 0           |
| Ninguno          | 2570  | 1176   | 13427   | 1065   | 105         |
| Bosque           | 95    | 0      | 2685    | 8335   | 385         |
| WOFS_Bosque      | 24    | 0      | 6       | 40     | 8           |

En el Modelo multinomial las clases ninguno y wofs_bosque tienen muy bajas estadisticas de presicion y recall por las siguentes razones: una razón de forma general para obtener tan bajas estadísticas usando el modelo multinomiál es que el trabaja mejor con datos descretos, es decir, valores que esten en un rango de predefinido, por el contrario los datos de características de nuestro dataset se encuentran en valores continuos.
En el modelo Gaussiano obtuvimos mejores características por la razón mencionada anteriormente. Sin embargo logramos apreciar que hay 2 clases(ninguno y wofs_bosque) que no estan teniendo buenas estadísticas de predicción(Precision y recall). Las razones a las que podemos llegar para esto son las siguientes: la clase wofs_bosque(Clase donde se categorizan los pixeles que se encuentran en un rango de valores que se pueden clasificar como Bosque y Agua) es una clase que se encuentra desbalanceada comparada con la cantidad de datos que otras clases tienen. Sin embargo como esta clase no es lo suficientemente determinante para nuestro ejercicio de clasificación decidimos dejarla de esta manera.
Por otro lado la clasificación “ninguno” puede estar presentando un bajo recall comparado con sus demas clases porque es una clasificación donde pueden estar una gran diversidad de valores haciendo que este modelo no generalice bien para esta clase particular.

# Referencias
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

https://datascience.stackexchange.com/questions/40345/how-to-convert-multiple-columns-into-single-columns-in-pandas

https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra